# Translation model from COCO -> MPII


In [59]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import json

In [33]:


ROOT_FOLDER = "/datagrid/personal/baljibil"
# ground truth
with open(ROOT_FOLDER + '/data/MPII_COCO/annotations/mpii_val.json', 'r') as f:
    annot_truth = json.load(f)
# prediction
with open(ROOT_FOLDER + '/repos/scripts/coco_pred_on_mpii/pred.json', 'r') as f:
    annot_pred_coco = json.load(f)

# with open(ROOT_FOLDER + '/repos/scripts/mpii_pred_on_mpii/pred.json', 'r') as f:
#     annot_pred_mpii = json.load(f)



In [63]:
X = list()
y = list()
box_scale = list()
box_center = list()
for i in range(len(annot_truth)):
  X.append(annot_pred_coco['annotations'][i]['joints'])
  y.append(annot_truth[i]['joints'])
  box_scale.append(annot_truth[i]['scale'])
  box_center.append(annot_truth[i]['center'])
X = np.array(X)
y = np.array(y)
box_scale = np.array(box_scale)
box_center = np.array(box_center)
print("X shape: ", X.shape)
print("y shape:", y.shape)
print("box scale:," box_scale.shape)
print("box center:", box_center.shape)

X shape:  (2958, 17, 2)
y shape: (2958, 16, 2)


## Normalize the dataset
- First by scale value of the bounding box
- Then either 0-1 normalization

In [53]:
from sklearn.model_selection import train_test_split

## Split train, val, test

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print("Train instance: ", X_train.shape[0])
print("Val instance: ", X_val.shape[0])
print("Test instance: ", X_test.shape[0])


Train instance:  1552
Val instance:  518
Test instance:  888


In [ ]:

class TranslatorNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(17, 17)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(17, 17)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(17, 16)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

model = PimaClassifier()
print(model)